In [1]:
import json
import os
import pandas as pd


def get_json_files():
    json_files = []
    for file in os.listdir("rawdata"):
        if file.endswith(".json"):
            json_files.append(file)
    return json_files


all_files = get_json_files()

In [2]:
data = pd.read_json("rawdata/" + all_files[1])

In [3]:
data.head()

,title,image_source,price,description,product_url
0,Get Well Sunshine Gift Hamper <br/>(Get Well S...,https://www.gifthampersuk.co.uk/image/cache/ca...,£50.00,Hamper ContentsAll Around The Fire Hot Chocola...,https://www.gifthampersuk.co.uk/care-hampers/g...
1,Recovery Treats Gift Hamper <br/>(Get Well Soo...,https://www.gifthampersuk.co.uk/image/cache/ca...,£45.00,Hamper Contents\r\nAlchemy Tea Rest and Recupe...,https://www.gifthampersuk.co.uk/care-hampers/r...
2,To Perk You Up Gift Hamper <br/>(Get Well Soon...,https://www.gifthampersuk.co.uk/image/cache/ca...,£43.00,Hamper Contents\r\nAnna’s Cappuccino Thins (15...,https://www.gifthampersuk.co.uk/care-hampers/a...
3,A Little Bit Of Calm Gift Hamper <br/>(Get Wel...,https://www.gifthampersuk.co.uk/image/cache/ca...,£60.00,Hamper Contents\r\nAlchemy Tea Howick Earl Gre...,https://www.gifthampersuk.co.uk/care-hampers/a...
4,Coffee Connoisseur's Dream Gift Hamper <br/>(G...,https://www.gifthampersuk.co.uk/image/cache/ca...,£50.00,Hamper ContentsGIFT ITEM: Life & Soul Cafetier...,https://www.gifthampersuk.co.uk/care-hampers/c...


In [4]:
from main import get_soup
from individualScraper import individual_product_info, dump_product_info

product_data_list_ = []
for index, row in data.iterrows():
    url = row["product_url"]
    soup = get_soup(url)
    product_info = individual_product_info(soup, product_url=url)
    product_data_list_.append(product_info)
    print(f"Scraped {index} of {len(data)}")

csv_file_name = "product_data.csv"
dump_product_info(product_data_list_, csv_file_name)

Scraped 0 of 9
Scraped 1 of 9
Scraped 2 of 9
Scraped 3 of 9
Scraped 4 of 9
Scraped 5 of 9


KeyboardInterrupt: 

In [18]:
from main import get_soup


def individual_product_info(soup, product_url):
    title = soup.find('h1', class_='product-name').text.strip()
    id_tag = soup.find('ul', id='brand-and-availability')
    ex_text = id_tag.find_all('span', class_='ex-text')
    availability = id_tag.find_all('li')[1].text.split(":")[1].strip()
    if len(ex_text) > 2:
        ex_text = id_tag.find_all('span', class_='ex-text')[1].text
        availability = id_tag.find_all('span', class_='ex-text')[2].text
    else:
        ex_text = id_tag.find_all('span', class_='ex-text')[0].text
        availability = id_tag.find_all('span', class_='ex-text')[1].text
    price_tag = soup.find('span', class_='placeholder-price-holder').text.strip()
    description_div = soup.find('div', id='tab-description')
    description_li = description_div.find_all('li')
    description = [desc.text for desc in description_li]
    if len(description) < 1:
        description_p = description_div.find_all('p')
        description = [desc.text for desc in description_p]

    image_src = soup.find("a", class_="thumbnail").img['src']

    product_data = {
        "product_id": ex_text,
        "title": title,
        "availability": availability,
        "price": price_tag,
        "description": description,
        "image_source": image_src,
        "product_url": product_url
    }

    return product_data


product_data_list_ = []
url = "https://www.gifthampersuk.co.uk/food-hampers/a-special-treat-for-you-gift-hamper-uk"
soup = get_soup(url)
product_info = individual_product_info(soup, url)
product_data_list_.append(product_info)

product_data_list_[0]

{'product_id': 'TGT: TGT-CORP-56 - White Wine Deli Luxury Hamper',
 'title': 'A Special Treat For You Gift Hamper',
 'availability': 'In Stock',
 'price': '£118.00',
 'description': ['Cotes de Gascogne Sauvignon Blanc (75cl)',
  'Cartwright & Butler Strawberry Jam (200g)',
  'Cartwright & Butler Finest Marmalade (200g)',
  'Sugar Soul Chocolate Covered Orange Peel (223g)',
  'James Patricks Liquorice Mix (190g)',
  'Cartwright & Butler Cranberry & Rolled Oats (200g)',
  'Alchemy Tea Howick Earl Grey, infuser and bag (approx. 30 cups)',
  'YOKOKO Raspberry Burst Luxury Dark Chocolate (100g)',
  'YOKOKO Lime & Chilli Luxury Dark Chocolate (100g)',
  'Luxury Gift Basket'],
 'image_source': 'https://www.gifthampersuk.co.uk/image/cache/catalog/products-ghuk/2023-gift-hampers/a-special-treat-for-you-gift-hamper-uk-651x800.jpg',
 'product_url': 'https://www.gifthampersuk.co.uk/food-hampers/a-special-treat-for-you-gift-hamper-uk'}